# Advanced Lane Finding Project

The goals / steps of this project are the following:
- Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- Apply a distortion correction to raw images.
- Use color transforms, gradients, etc., to create a thresholded binary image.
- Apply a perspective transform to rectify binary image ("birds-eye view").
- Detect lane pixels and fit to find the lane boundary.
- Determine the curvature of the lane and vehicle position with respect to center.
- Warp the detected lane boundaries back onto the original image.
- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## 0. Imports

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

---
## 1. Camera Calibration

Camera calibration only needs to occur once, but I have put it in a function to encourage good coding behavior

In [ ]:
def find_checkerboard_corners(img, nx, ny):
    """Helper function to detect the internal corners of an image of a checkerboard
    
    Parameters:
        • img - image of a checkerboard
        • nx - number of internal checkerboard corners in the x direction
        • ny - number of internal checkerboard corners in the y direction
        
    Returns:
        A list of corners"""
    
    # openCV reads in images BGR (instead of RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    if ret:
        return corners
    
    return []


def camera_calibration(img_files, nx=9, ny=6):
    """Calibrate the camera based on several checkerboard images taken.
    
    Parameters:
        • img_files - list of checkboard image files, ideally taken from different angles and distances
        • nx - number of internal checkerboard corners in the x direction for all images
        • ny - number of internal checkerboard corners in the y direction for all images
        
    Returns:
        Tuple of the camera matrix and the distortion coefficients"""
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((ny * nx, 3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    for idx, img_file in enumerate(img_files):
        img = cv2.imread(img_file)
        
        # find the checkerboard corners
        corners = find_checkerboard_corners(img, nx, ny)

        # If found, add object points, image points (after refining them)
        if len(corners):
            objpoints.append(objp)
            imgpoints.append(corners)
    
    img_size = (img.shape[1], img.shape[0])
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    
    return (mtx, dist)


In [ ]:
img_files = glob.glob('camera_cal/*.jpg')
mtx, dist = camera_calibration(img_files)

img_file = img_files[13]

img = cv2.imread(img_file)
cv2.imwrite('output_images/01-checkerboard-orig.jpg', img)

plt.title('Original Image of a Checkerboard')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

corners = find_checkerboard_corners(img, 9, 6)
cv2.drawChessboardCorners(img, (9, 6), corners, True)
cv2.imwrite('output_images/02-checkerboard-corners.jpg', img)

fig = plt.figure()
plt.title('Detected Corners on a Checkerboard')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB));


---
## 2. Undistort

This will need to be run on all images produced by the camera that was calibrated in the previous step

In [ ]:
def undistort(img, mtx, dist):
    """Undistort an image using the camera matrix and the distorition coefficients returned by the 'camera_alibration'
    function
    
    Parameters:
        • img - image that needs to be undistorted
        • mtx - camera matrix
        • dist - distortion coefficients
        
    Returns:
        An undistorted image"""
    
    img_size = (img.shape[1], img.shape[0])
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist


In [ ]:
img = cv2.imread(img_files[13])

plt.title('Original Image of a Checkerboard')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

img = undistort(img, mtx, dist)
cv2.imwrite('output_images/03-checkerboard-undistorted.jpg', img)

fig = plt.figure()
plt.title("Undistorted Image")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB));

In [ ]:
undistort_imgs = []
file_bases = [f.split('/')[-1][:-4] for f in glob.glob('test_images/*.jpg')]

for filename, file_base in zip(glob.glob('test_images/*.jpg'), file_bases):
    
    orig_img = cv2.imread(filename)
    cv2.imwrite('output_images/{}-orig-00.jpg'.format(file_base), orig_img)
    
    undistort_img = undistort(orig_img, mtx, dist)
    cv2.imwrite('output_images/{}-undist-01.jpg'.format(file_base), undistort_img)

    undistort_imgs.append(undistort_img)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Original Image')
    ax1.imshow(cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB))

    ax2.set_title('Undistorted Image')
    ax2.imshow(cv2.cvtColor(undistort_img, cv2.COLOR_BGR2RGB))

---
## 3. Thresholding functions

Various thresholding functions determine lane boundries. To make the algorithm more robust, these will be combined together.

In [ ]:
def absolute_sobel_threshold(img, orient='x', sobel_kernel=3, thresh=(0, 255)):

    """Creates a binary image based on the gradient ine one direction of the input image.
    
    This function uses the Sobel operator to calculate the derivative.
    
    Parameters:
        • img - input image
        • orient - direction to take the derivative ('x' or 'y')
        • sobel_kernel - an odd number to define the size of the Sobel kernel
        • thresh - tuple of low and high thresholds to be included in the output binary
        
    Returns:
        A single channel binary image of detected edges in the original image"""
    
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    filter_type = (orient == 'x', orient == 'y')
    sobel = cv2.Sobel(gray, cv2.CV_64F, *filter_type, ksize=sobel_kernel)

    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)

    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))

    # 5) Create a mask of 1's where the scaled gradient meets the thresholds 
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # 6) Return this mask as your binary_output image
    return grad_binary



In [ ]:
def magnitude_threshold(img, sobel_kernel=3, mag_thresh=(0, 255)):

    """Creates a binary image based on the overal magnitude of the gradient of the input image.
    
    This function uses the Sobel operator to calculate the derivative.
    
    Parameters:
        • img - input image
        • sobel_kernel - an odd number to define the size of the Sobel kernel
        • thresh - tuple of low and high thresholds to be included in the output binary
        
    Returns:
        A single channel binary image of detected edges in the original image"""

    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # 3) Calculate the magnitude 
    abs_sobelxy = (sobelx ** 2 + sobely ** 2) ** 0.5

    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255 * abs_sobelxy / np.max(abs_sobelxy))

    # 5) Create a binary mask where mag thresholds are met
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1

    # 6) Return this mask as your binary_output image
    return mag_binary


In [ ]:
def direction_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):

    """Creates a binary image based on the gradient direction of the input image.
    
    This function uses the Sobel operator to calculate the derivative.
    
    Parameters:
        • img - input image
        • sobel_kernel - an odd number to define the size of the Sobel kernel
        • thresh - tuple of low and high thresholds to be included in the output binary
        
    Returns:
        A single channel binary image of detected edges in the original image"""

    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)

    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    direction = np.arctan2(abs_sobely, abs_sobelx)

    # 5) Create a binary mask where direction thresholds are met
    dir_binary = np.zeros_like(direction)
    dir_binary[(direction >= thresh[0]) & (direction <= thresh[1])] = 1

    # 6) Return this mask as your binary_output image
    return dir_binary

In [ ]:
def saturation_threshold(img, thresh=(0, 255)):
    
    """Creates a binary image based on the saturation channel of the input image converted to HLS color space.
        
    Parameters:
        • img - input image
        • thresh - tuple of low and high thresholds to be included in the output binary
        
    Returns:
        A thresholded single channel binary image of the original image"""

    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    # 2) Apply a threshold to the S channel
    s = hls[:, :, 2]
    sat_binary = np.zeros_like(s)
    sat_binary[(s > thresh[0]) & (s <= thresh[1])] = 1

    # 3) Return a binary image of threshold result
    return sat_binary


In [ ]:
def threshold(img):
    
    """Creates a binary image based on combining various thresholding techniques.
    
    Paramters:
        • img - input image
        
    Returns:
        A binary image where the lane markers are clearly visible"""
    
    ksize = 5
    
    gradx = absolute_sobel_threshold(img, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    grady = absolute_sobel_threshold(img, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = magnitude_threshold(img, sobel_kernel=ksize, mag_thresh=(20, 100))
    dir_binary = direction_threshold(img, sobel_kernel=ksize, thresh=(0.7, 1.3))
    
    sat_binary = saturation_threshold(img, thresh=(90, 255))

    combined = np.zeros_like(dir_binary)
    combined[(((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))) | (sat_binary == 1)] = 1

    return combined

In [ ]:
thresholded_imgs = []

for undistort_img, file_base in zip(undistort_imgs, file_bases):

    thresholded_img = threshold(undistort_img)
    cv2.imwrite('output_images/{}-thresh-02.jpg'.format(file_base), thresholded_img)
    
    thresholded_imgs.append(thresholded_img)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Undistorted Image')
    ax1.imshow(cv2.cvtColor(undistort_img, cv2.COLOR_BGR2RGB))

    ax2.set_title('Thresholded Image')
    ax2.imshow(thresholded_img, cmap='gray')

---

## 4. Perspective Transformation

These functions will transform images from perspective view to birds-eye view

In [ ]:
def transform_birdseye(img):
    
    """Applies a transformation to warp an image. The result is a birds-eye view of the lane.
    
    Parameters:
        • img - input image of the lane
        
    Returns:
        A birds-eye view image of the lane"""
    
    # define the source and destination points for the desired transformation
    src = np.float32([[190, 720], [596, 446], [692, 446], [1140, 720]])    
    dst = np.float32([[190, 720], [190, 0], [1140, 0], [1140, 720]])
    
    # get the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    
    # get the image size
    img_size = (img.shape[1], img.shape[0])
    
    # warp the image
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped

In [ ]:
birdseye_imgs = []

for thresholded_img, file_base in zip(thresholded_imgs, file_bases):

    birdseye_img = transform_birdseye(thresholded_img)
    cv2.imwrite('output_images/{}-warp-03.jpg'.format(file_base), birdseye_img)
    
    birdseye_imgs.append(thresholded_img)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Thresholded Image')
    ax1.imshow(thresholded_img, cmap='gray')

    ax2.set_title('Birds-Eye Warped Image')
    ax2.imshow(birdseye_img, cmap='gray')